In [80]:
import tensorflow as tf
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.applications.xception import Xception
import segmentation_models as sm
import tensorflow_advanced_segmentation_models as tasm
import numpy as np

## Failed ResNet Test

In [2]:
resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(512, 512, 3))

In [10]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [28]:
layer_names = ["conv1_pad", "conv1_conv", "conv1_bn", "conv1_relu", "pool1_pad" "pool1_pool"]

resnet_blocks = {"conv7": ["conv1_pad", "conv1_conv", "conv1_bn", "conv1_relu", "pool1_pad", "pool1_pool"],
                 "block_3" : ["conv2_block3_1_conv", "conv2_block3_1_bn", "conv2_block3_1_relu", "conv2_block3_2_conv", "conv2_block3_2_bn", "conv2_block3_2_relu"],
                 "block_4" : ["conv3_block4_1_conv", "conv3_block4_1_bn", "conv3_block4_1_relu", "conv3_block4_2_conv", "conv3_block4_2_bn", "conv3_block4_2_relu"],
                 "block_6" : ["conv4_block6_1_conv", "conv4_block6_1_bn", "conv4_block6_1_relu", "conv4_block6_2_conv", "conv4_block6_2_bn", "conv4_block6_2_relu"],
                 "block_3_2" : ["conv5_block3_1_conv", "conv5_block3_1_bn", "conv5_block3_1_relu", "conv5_block3_2_conv", "conv5_block3_2_bn", "conv5_block3_2_relu"]}

In [47]:
layers = [resnet.get_layer(layer_name) for layer_name in resnet_blocks["block_3"]]

In [45]:
x = tf.keras.layers.Input((512, 512, 3))
copy = x
for layer in layers:
    x = layer(x)
#output = tf.keras.layers.Add()([copy, x])

In [46]:
x.shape

TensorShape([None, 128, 128, 64])

In [48]:
for layer in layers:
    x = layer(x)

ValueError: Exception encountered when calling layer "conv2_block3_1_conv" (type Conv2D).

Depth of input (64) is not a multiple of input depth of filter (256) for '{{node conv2_block3_1_conv/Conv2D}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](Placeholder, conv2_block3_1_conv/Conv2D/ReadVariableOp)' with input shapes: [?,128,128,64], [1,1,256,64].

Call arguments received by layer "conv2_block3_1_conv" (type Conv2D):
  • inputs=tf.Tensor(shape=(None, 128, 128, 64), dtype=float32)

## Xception

In [57]:
x = Xception(weights="imagenet", input_shape=(512,512, 3), include_top=False)

In [58]:
x.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_18 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 255, 255, 32  864         ['input_18[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 255, 255, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                          

## VGG U-Net

In [52]:
vgg19 = VGG19(weights="imagenet", include_top=False, input_shape=(512,512,3))

In [53]:
vgg19.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_20 (InputLayer)       [(None, 512, 512, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 512, 512, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 512, 512, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 256, 256, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 256, 256, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 256, 256, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 128, 128, 128)     0     

In [93]:
#Getting all the blocks from the VGG network
vgg_blocks = {
    f"block{n}" : [layer for layer in vgg19.layers if f"block{n}_conv" in layer.name] for n in range(1, 6)
}

In [101]:
vgg_blocks

{'block1': [<keras.layers.convolutional.conv2d.Conv2D at 0x7f920b32ca50>,
 'block2': [<keras.layers.convolutional.conv2d.Conv2D at 0x7f920cc915d0>,
 'block3': [<keras.layers.convolutional.conv2d.Conv2D at 0x7f92091997d0>,
 'block4': [<keras.layers.convolutional.conv2d.Conv2D at 0x7f920b3cd410>,
 'block5': [<keras.layers.convolutional.conv2d.Conv2D at 0x7f920b3ead50>,
  <keras.layers.convolutional.conv2d.Conv2D at 0x7f920b458dd0>]}

### VGG 1 - Encoder

In [110]:
def vgg_encoder_block(x, layers):
    """
    This function passes an input through a set of conv layers from VGG19, returning the downsampled and convolved activation
    """
    for layer in layers:
        x = layer(x)
    
    addition = x
    x = tf.keras.layers.MaxPooling2D((2,2), strides = 2)(x)
    return (x, addition)

def last_vgg_block(x, layers):

    for layer in layers:
        x = layer(x)
    
    return x

In [75]:
input_x = vgg19.input

In [76]:
input_x

<KerasTensor: shape=(None, 512, 512, 3) dtype=float32 (created by layer 'input_20')>

In [72]:
block_1 = vgg_encoder_block(input_x, vgg_blocks["block1"])

In [73]:
block_1

(<KerasTensor: shape=(None, 256, 256, 64) dtype=float32 (created by layer 'max_pooling2d')>,
 <KerasTensor: shape=(None, 512, 512, 64) dtype=float32 (created by layer 'block1_conv2')>)

In [111]:
def vgg_encoder_full(input, layer_dict):

    """
    This function creates the full encoder given a dictionary of layers from the VGG network, it returns the final activation 
    and a list of intermediate activations
    """

    activations = []
    x = input
    for layer_name in list(layer_dict.keys())[:-1]:
        x, a = vgg_encoder_block(x, layer_dict[layer_name])
        activations.append(a)
    
    x = last_vgg_block(x, layer_dict[list(layer_dict.keys())[-1]])
    
    return x, activations

In [112]:
x, a = vgg_encoder_full(input_x, vgg_blocks)

In [113]:
x

<KerasTensor: shape=(None, 32, 32, 512) dtype=float32 (created by layer 'block5_conv4')>

### VGG 2 - Decoder

In [91]:
def decoder_block(a, x, f):

    x = tf.keras.layers.Conv2DTranspose(filters=f, kernel_size=2, strides=2, padding="same", activation="relu")(x)
    x = tf.concat([a, x], axis=-1)
    x = tf.keras.layers.Conv2D(f, 3, padding="same", activation="relu")(x)
    x = tf.keras.layers.Conv2D(f, 3, padding="same", activation="relu")(x) 

    return x

In [96]:
def decoder_full(activations, x, filters, num_classes):

    for a,f in zip(activations[::-1],filters):
        x = decoder_block(a, x, f)
    
    output = tf.keras.layers.Conv2D(num_classes, 1, padding="same", activation="softmax")(x)

    return output

In [114]:
filters = [512, 256, 128, 64]
o = decoder_full(a, x, filters, 3)

In [115]:
o

<KerasTensor: shape=(None, 512, 512, 3) dtype=float32 (created by layer 'conv2d_17')>

### VGG 3 - Full U-Net


In [128]:
def vgg_unet(num_classes, input_size, input_dim):

    #Downloading the VGG network
    vgg19 = VGG19(weights="imagenet", include_top=False, input_shape=(input_size, input_size,input_dim))
    vgg19.trainable = False
    #Getting all the blocks from the VGG network
    vgg_blocks = {
        f"block{n}" : [layer for layer in vgg19.layers if f"block{n}_conv" in layer.name] for n in range(1, 6)
    }
    
    #Filters for the Decoder
    filters = [512, 256, 128, 64]

    vgg_input = vgg19.input

    #Defining the encoder

    #First Preprocess the input
    x = preprocess_input(x=vgg_input)
    
    x, a = vgg_encoder_full(x, vgg_blocks)

    output = decoder_full(a, x, filters, num_classes)

    vgg_unet_model = tf.keras.Model(vgg_input, output)

    return vgg_unet_model
    

    

    

In [133]:
vnet = vgg_unet(3, 512, 3)

In [132]:
vnet.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_26 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 tf.__operators__.getitem_4 (Sl  (None, 224, 224, 3)  0          ['input_26[0][0]']               
 icingOpLambda)                                                                                   
                                                                                                  
 tf.nn.bias_add_4 (TFOpLambda)  (None, 224, 224, 3)  0           ['tf.__operators__.getitem_4[0][0
                                                                 ]']                        

## ResNet-U-Net

In [2]:
resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(512,512,3))

In [3]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [94]:
resnet_blocks = {
    f"block_{n}" : [layer for layer in resnet.layers if layer.name[:5] == f"conv{n}"] for n in range(1, 5)
}

In [92]:
resnet_outputs = {
    f"output_{n}" : [layer.name for layer in resnet.layers if layer.name[:5] == f"conv{n}" and "out" in layer.name][-1] for n in range(2, 5)
}

In [56]:
size = 0
for k in resnet_blocks.keys():
    size += len(resnet_blocks[k])
print(f"Number of layers {size}")

Number of layers 140


In [8]:
su = [1 for layer in resnet.layers]

In [10]:
sum(su)

175

### Encoder of the ResNet-U-Net

In [ ]:
resnet_blocks

In [123]:
def resblock_stem(input, layers):

    x = input
    for layer in layers:

        x = layer(x)
    
    a = x
    x = tf.keras.layers.MaxPool2D(2, 2)(x)

    return x, a


def resblock_enc(inp, layers):

    enc = tf.keras.Model(inputs=resnet.get_layer(layers[0].name).input, outputs=resnet.get_layer(layers[-1].name).output)
    x = enc(inp)
    return x

def resnet_encoder(inp, layer_dict):

    keys = list(layer_dict.keys())

    x, a = resblock_stem(inp, layer_dict[keys[0]])

    activations = [a]
    for block in keys[1:]:
        x = resblock_enc(x, layer_dict[block]) 
        activations.append(x)
    
    return x, [None] + activations[:-1]



In [108]:
inp = resnet.input

In [109]:
x, a = resblock_stem(inp, resnet_blocks["block_1"])

In [110]:
x

<KerasTensor: shape=(None, 128, 128, 64) dtype=float32 (created by layer 'max_pooling2d_5')>

In [111]:
x2 = resblock_enc(x, resnet_blocks["block_2"])

In [124]:
x2

<KerasTensor: shape=(None, 128, 128, 256) dtype=float32 (created by layer 'model_2')>

In [129]:
x, a = resnet_encoder(inp, resnet_blocks)

### Decoder for The ResNet-Unet

The last decoder block of this U-Net will not use an activation

In [126]:
def decoder_block(a, x, f):

    x = tf.keras.layers.Conv2DTranspose(filters=f, kernel_size=2, strides=2, padding="same", activation="relu")(x)
    if a is not  None:
        x = tf.concat([a, x], axis=-1)
    x = tf.keras.layers.Conv2D(f, 3, padding="same", activation="relu")(x)
    x = tf.keras.layers.Conv2D(f, 3, padding="same", activation="relu")(x) 

    return x

def decoder_full(activations, x, filters, num_classes):

    for a,f in zip(activations[::-1],filters):
        x = decoder_block(a, x, f)
    
    output = tf.keras.layers.Conv2D(num_classes, 1, padding="same", activation="softmax")(x)

    return output

In [127]:
filters = [512, 256, 128, 64]

In [130]:
o = decoder_full(a, x, filters, 5)

### Full ResNet-Unet

In [155]:
def resblock_stem(input, layers):

    x = input
    for layer in layers:

        x = layer(x)
    
    a = x
    x = tf.keras.layers.MaxPool2D((2,2), 2)(x)

    return x, a


def resblock_enc(inp, layers, res):

    enc = tf.keras.Model(inputs=res.get_layer(layers[0].name).input, outputs=res.get_layer(layers[-1].name).output)
    x = enc(inp)
    return x

def resnet_encoder(inp, layer_dict, res):

    keys = list(layer_dict.keys())

    x, a = resblock_stem(inp, layer_dict[keys[0]])

    activations = [a]
    for block in keys[1:]:
        x = resblock_enc(x, layer_dict[block], res) 
        activations.append(x)
    
    return x, [None] + activations[:-1]



def resnet_unet(num_classes, input_size, input_dim):

    #Downloading the ResNet
    resnet = ResNet50(weights="imagenet", include_top=False, input_shape=(input_size,input_size,input_dim))

    layer_dict = resnet_blocks = {
    f"block_{n}" : [layer for layer in resnet.layers if layer.name[:5] == f"conv{n}"] for n in range(1, 5)
    }

    #Freezing the layers of the ResNet
    resnet.trainable = False

    #Building the model
    inp = resnet.input
    x = preprocess_input(x=inp)
    x, a = resnet_encoder(inp, layer_dict, resnet)

    filters = [512, 256, 128, 64]

    output = decoder_full(a, x, filters, num_classes)

    model = tf.keras.Model(inp, output)

    return model

In [156]:
m = resnet_unet(5, 512, 3)

In [152]:
m.summary()

Model: "model_41"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_38 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_38[0][0]']               
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[1][0]']              
                                )                                                                 
                                                                                           

## Efficient-Net

In [157]:
from tensorflow.keras.applications.efficientnet import EfficientNetB4

In [160]:
b4 = EfficientNetB4(weights="imagenet", include_top=False, input_shape=(512,512,3))

71686520/71686520 [==============================] - 5s 0us/step


In [161]:
b4.summary()

Model: "efficientnetb4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_52 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_4 (Rescaling)        (None, 512, 512, 3)  0           ['input_52[0][0]']               
                                                                                                  
 normalization_2 (Normalization  (None, 512, 512, 3)  7          ['rescaling_4[0][0]']            
 )                                                                                                
                                                                                     

## Base U-Net

In [178]:
def encoder_block(inp,f):

    x = inp

    x = tf.keras.layers.Conv2D(f, 3, 1, padding="same")(x)
    x = tf.keras.layers.Conv2D(f, 3, 1, padding="same")(x)
    a = x
    x = tf.keras.layers.MaxPool2D(2, 2)(x)

    return x,a

def last_encoder(inp, f):

    x = inp

    x = tf.keras.layers.Conv2D(f, 3, 1, padding="same")(x)
    x = tf.keras.layers.Conv2D(f, 3, 1, padding="same")(x)

    return x

In [179]:
def encoder_unet(inp, filters):

    activations = []

    x = inp
    for f in filters[:-1]:
        x, a = encoder_block(x, f)
        activations.append(a)
    
    x = last_encoder(x, filters[-1])
    return x, activations

In [180]:
encoder_unet(tf.keras.layers.Input((512, 512, 3)), [64, 128, 256, 512, 1024])

(<KerasTensor: shape=(None, 32, 32, 1024) dtype=float32 (created by layer 'conv2d_111')>,
 [<KerasTensor: shape=(None, 512, 512, 64) dtype=float32 (created by layer 'conv2d_103')>,
  <KerasTensor: shape=(None, 256, 256, 128) dtype=float32 (created by layer 'conv2d_105')>,
  <KerasTensor: shape=(None, 128, 128, 256) dtype=float32 (created by layer 'conv2d_107')>,
  <KerasTensor: shape=(None, 64, 64, 512) dtype=float32 (created by layer 'conv2d_109')>])

In [181]:
def unet(num_classes, input_size, input_dim):

    inp = tf.keras.layers.Input((input_size, input_size, input_dim))

    filters = [64, 128, 256, 512, 1024]

    x, a = encoder_unet(inp, filters=filters)

    o = decoder_full(a, x, filters, num_classes)

    model = tf.keras.Model(inp, o)

    return model

In [182]:
un = unet(3, 512, 3)

In [183]:
un.summary()

Model: "model_49"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_60 (InputLayer)          [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_112 (Conv2D)            (None, 512, 512, 64  1792        ['input_60[0][0]']               
                                )                                                                 
                                                                                                  
 conv2d_113 (Conv2D)            (None, 512, 512, 64  36928       ['conv2d_112[0][0]']             
                                )                                                          